In [1]:
import pandas as pd
import numpy as np

In [7]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84406 entries, 0 to 84405
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       84406 non-null  object 
 1   월        84406 non-null  int64  
 2   요일       84406 non-null  object 
 3   시간       84406 non-null  int64  
 4   소관경찰서    84406 non-null  int64  
 5   소관지역     84406 non-null  float64
 6   사건발생거리   84406 non-null  float64
 7   강수량(mm)  84406 non-null  float64
 8   강설량(mm)  84406 non-null  float64
 9   적설량(cm)  84406 non-null  float64
 10  풍향       84406 non-null  float64
 11  안개       84406 non-null  float64
 12  짙은안개     84406 non-null  float64
 13  번개       84406 non-null  float64
 14  진눈깨비     84406 non-null  float64
 15  서리       84406 non-null  float64
 16  연기/연무    84406 non-null  float64
 17  눈날림      84406 non-null  float64
 18  범죄발생지    84406 non-null  object 
 19  TARGET   84406 non-null  int64  
dtypes: float64(13), int64(4), object(3)
memory usage: 

In [17]:
train.head()

,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,TRAIN_00000,9,화요일,10,137,8.0,2.611124,0.000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,TRAIN_00001,11,화요일,6,438,13.0,3.209093,0.000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,TRAIN_00002,8,일요일,6,1729,47.0,1.619597,0.000,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,TRAIN_00003,5,월요일,6,2337,53.0,1.921615,11.375,0.0,0.0,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,TRAIN_00004,9,일요일,11,1439,41.0,1.789721,0.000,0.0,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2


In [ ]:
import seaborn as sns
sns.heatmap(train.corr(),
            annot = False)



In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train['요일'] = le.fit_transform(train['요일'])
train['범죄발생지'] = le.fit_transform(train['범죄발생지'])

In [49]:
from sklearn.model_selection import train_test_split

x = train.drop(['ID','TARGET'], axis = 1)
y = train.loc[:,'TARGET']

x_train, x_val, y_train, y_val = train_test_split(x,y, test_size = 0.2, random_state = 0)



In [50]:
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier()

modelRF.fit(x_train, y_train)

RandomForestClassifier()

In [51]:
from sklearn.metrics import *
y_pred = modelRF.predict(x_val)

print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))


[[4932 1179 1191]
 [1910 2328  829]
 [2118  855 1540]]
              precision    recall  f1-score   support

           0       0.55      0.68      0.61      7302
           1       0.53      0.46      0.49      5067
           2       0.43      0.34      0.38      4513

    accuracy                           0.52     16882
   macro avg       0.51      0.49      0.49     16882
weighted avg       0.51      0.52      0.51     16882



In [ ]:
#pip install mljar-supervised

In [ ]:
from supervised.automl import AutoML
automl = AutoML(mode="Explain")
automl.fit(x_train, y_train)

In [38]:

y_pred = automl.predict(x_val)

print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))


[[5311  888 1103]
 [2019 2221  827]
 [2065  708 1740]]
              precision    recall  f1-score   support

           0       0.57      0.73      0.64      7302
           1       0.58      0.44      0.50      5067
           2       0.47      0.39      0.43      4513

    accuracy                           0.55     16882
   macro avg       0.54      0.52      0.52     16882
weighted avg       0.55      0.55      0.54     16882



In [41]:
test['요일'] = le.fit_transform(test['요일'])
test['범죄발생지'] = le.fit_transform(test['범죄발생지'])

In [42]:
y_pred = automl.predict(test)

In [48]:
from catboost import CatBoostClassifier, cv
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier

In [ ]:
estimators = [('rf', RandomForestClassifier(max_depth = 9)),
              ('cat', CatBoostClassifier()),
              ('lr', LogisticRegression(max_iter=1000)),
              ('xgb', XGBClassifier(max_depth = 5)),
              ('knn', KNeighborsClassifier()),
              ('dt', DecisionTreeClassifier(max_depth = 7)),
              ('lgbm', LGBMClassifier())]

modelST = StackingClassifier(estimators=estimators,
                           final_estimator= RandomForestClassifier())


# 학습하기
modelST.fit(x_train, y_train)

# 예측하기
y_pred = modelST.predict(x_val)

# 평가하기
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))

In [55]:
sub_label = pd.Series(y_pred)
sub = pd.read_csv("sample_submission.csv")
sub['TARGET'] = sub_label
sub = sub.set_index('ID')
sub.to_csv("C:/Users/User/Desktop/Dacon/crime/submission.csv")